# 1. Extracción de imágenes de Google

In [ ]:
import hashlib
from PIL import Image
import io, os
import requests
import time
import selenium
from selenium import webdriver

In [ ]:
wd = webdriver.Chrome('chromedriver.exe')

In [ ]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=2):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        print(image_count,max_links_to_fetch)
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        
        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [ ]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.1)
        
    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
search_term = 'patinete'
search_and_download(search_term,'chromedriver.exe',number_images=100)

In [ ]:
search_term = 'bicicleta'
search_and_download(search_term,'chromedriver.exe',number_images=100)

# 2. Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import os
from os.path import isfile, join
import pandas as pd
import numpy as np
from skimage.io import imread

In [ ]:
# 1. Iniciar el data generator
datagen = ImageDataGenerator(rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.15,
                            zoom_range=0.1,
                            channel_shift_range=10,
                            horizontal_flip=True)

In [ ]:
# 2. Aplicar las transfromaciones en una imagen
patinetes = ['images/patinete/'+ f for f in os.listdir('images/patinete') if isfile(join('images/patinete', f))]
image_path = patinetes[0]
image = np.expand_dims(imread(image_path),axis=0)

In [ ]:
# 3. Selecciona dónde quieres guardar las imágenes transformadas
save_here = 'images/patinete/aug'
try:
    os.stat(save_here)
except:
    os.mkdir(save_here)

In [ ]:
# Fit de la imagen original
datagen.fit(image)

In [ ]:
for x, val in zip(datagen.flow(image,           #image we chose
        save_to_dir=save_here,     #this is where we figure out where to save
         save_prefix='aug',        # it will save the images as 'aug_0912' some number for every new augmented image
        save_format='png'),range(10)):     # here we define a range because we want 10 augmented images 
    pass

In [ ]:
# Para todos las imágenes de la clase 1
for image_path in patinetes:
    image = np.expand_dims(imread(image_path),axis=0)
    datagen.fit(image)
    for x, val in zip(datagen.flow(image,                    #image we chose
        save_to_dir=save_here,     #this is where we figure out where to save
         save_prefix='aug',        # it will save the images as 'aug_0912' some number for every new augmented image
        save_format='png'),range(3)):     # here we define a range because we want 10 augmented images 
        pass

In [ ]:
# Realiza la misma operación para las bicicletas
bicicletas = ['images/bicicleta/'+ f for f in os.listdir('images/bicicleta') if isfile(join('images/bicicleta', f))]
save_here = 'images/bicicleta/aug'
try:
    os.stat(save_here)
except:
    os.mkdir(save_here)  
    
for image_path in bicicletas:
    image = np.expand_dims(imread(image_path),axis=0)
    datagen.fit(image)
    for x, val in zip(datagen.flow(image,                    #image we chose
        save_to_dir=save_here,     #this is where we figure out where to save
         save_prefix='aug',        # it will save the images as 'aug_0912' some number for every new augmented image
        save_format='png'),range(3)):     # here we define a range because we want 10 augmented images 
        pass

# 3. Preparar los datos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

In [ ]:
# Read data
X = []
Y = []
for file in os.listdir('images/patinete/aug'):
    image = imread('images/patinete/aug/'+file)
    smallimage = cv2.resize(image,(64,64))
    X.append(smallimage)
    Y.append(0)
    
for file in os.listdir('images/bicicleta/aug'):
    image = imread('images/bicicleta/aug/'+file)
    smallimage = cv2.resize(image,(64,64))
    X.append(smallimage)
    Y.append(1)

In [ ]:
X,Y = np.array(X), np.array(Y)

In [ ]:
X = X/255.0
print('Min:',np.min(X))
print('Max:',np.max(X))

In [ ]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, 
                                                    random_state=40)
print(X_train.shape); print(X_test.shape)

#  Transfer Learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
base_model = VGG16(input_shape=(64,64,3),
                  include_top = False,
                   weights = 'imagenet'
                  )

In [ ]:
base_model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output) # Añado una flatten a la salida del base_model, al cual le quité el top CAPA(CAPA ANTERIOR)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x) # Añado una densa a x (flatten)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = model.fit(X_train,y_train,
                    epochs = 10)

In [ ]:
results = model.evaluate(X_test, y_test)
print("test loss, test accuracy", results)

In [ ]:
plt.imshow(X_test[3])

In [ ]:
model.predict(np.expand_dims(X_test[3], axis=0))